In [15]:
#import 
import os
import logging
import time
import csv
import html
import jinja2
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import uuid
import yaml
import requests
import io
import json
import re
import dateutil.parser

In [18]:
csv_test = "/home/sdanioth/Documents/git/oscar-catalogue/KCSAP_AWS.csv"

sanitize = lambda x: html.escape(x) if len(x) > 0 else 'unknown'

with open(csv_test) as f:
    reader = csv.DictReader( f )
    files = []
    for row in reader:
        var = 'WSI'
        d1 = { key: sanitize(row[key]) for key in row.keys() if key != var }
        #print(d1)
        for ele in row[var].split(','):
            csv_test = {var: ele.strip(), 'uuid': uuid.uuid4()}
            csv_test.update(d1)
            #print(d1['DateEstablished'])

            d1['DateEstablished'] = dateutil.parser.parse(d1['DateEstablished'],fuzzy=True).date()
            print(d1['DateEstablished'])

2021-09-16
2021-09-17
2021-09-19
2021-09-20
2021-12-22
2021-01-10
2021-11-12
2021-01-10
2021-05-10
2021-08-10
2021-10-14
2021-10-13
2021-10-18
2021-10-19
2021-10-24
2021-10-26
2021-10-27
2021-01-11
2021-03-11
2021-10-28
2021-03-11
2021-10-11
2021-10-11
2021-11-15
2021-11-15
2021-11-16
2021-11-16
2021-11-11
2021-12-11
2021-11-17
2021-11-18
2021-11-17
2021-11-18
2021-11-20
2021-11-19
2021-11-21
2021-11-21
2021-11-22
2021-11-22
2023-11-23
2021-12-21
2021-11-25
2021-11-24
2021-11-24
2021-11-29
2021-11-27
2021-11-29
2021-11-28
2021-11-29
2021-03-12
2021-03-12
2021-05-12
2021-05-12
2021-06-12
2021-06-12
2021-09-12
2021-10-12
2021-09-12
2021-10-12
2021-07-12
2021-12-16
2021-12-17
2021-12-15
2021-12-17
2022-12-01
2022-01-17
2022-01-17
2022-01-18
2022-01-20
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-24
2022-01-24
2022-01-30
2022-01-30
2022-01-31


In [32]:
config = "/home/sdanioth/Documents/git/oscar-catalogue/config.yaml"
with open(os.path.abspath(config), "r") as f:
    config = yaml.safe_load(f)
    f.close()


In [33]:
for var in config['variablesWMDR']:
    print(var)

ObservedVariableAtmosphere


In [34]:
label = "ObservedVariableAtmosphere"
print(label)

ObservedVariableAtmosphere


In [127]:
facility = os.path.expanduser(config['FacilitySource'])


In [128]:
sanitize = lambda x: html.escape(x) if len(x) > 0 else 'unknown'

with open(facility) as f:
    reader = csv.DictReader( f )
    files = []
    for row in reader:
        var = 'WIGOSstationIdentifier'
        d1 = { key: sanitize(row[key]) for key in row.keys() if key != var }
        for ele in row[var].split(','):
            facility = {var: ele.strip(), 'uuid': uuid.uuid4()}
            facility.update(d1)

In [129]:
facility["Name"]

'Test Multiple variables'

In [130]:
csv_number = facility["ObservedVariableAtmosphere"]
print(csv_number)




Air temperature (at specified distance from reference surface), Visibility, Horizontal wind direction at specified distance from reference surface, Horizontal wind speed at specified distance from reference surface, Relative humidity (with respect to water), Atmospheric pressure, Amount of precipitation
A


In [134]:
test = csv_number.split(",")
print(test)
print(test[4])

['Air temperature (at specified distance from reference surface)', ' Visibility', ' Horizontal wind direction at specified distance from reference surface', ' Horizontal wind speed at specified distance from reference surface', ' Relative humidity (with respect to water)', ' Atmospheric pressure', ' Amount of precipitation']
 Relative humidity (with respect to water)


In [135]:
variables = facility["ObservedVariableAtmosphere"]
variables = variables.split(",")

In [136]:
for var in variables:
    print(var)

Air temperature (at specified distance from reference surface)
 Visibility
 Horizontal wind direction at specified distance from reference surface
 Horizontal wind speed at specified distance from reference surface
 Relative humidity (with respect to water)
 Atmospheric pressure
 Amount of precipitation


In [2]:
token = '424bdd5a-df56-430b-937a-9e48957cea99'
file = '/home/sdanioth/Documents/git/oscar-catalogue/oscar-facility-catalogue/test_survey_2_KEN_234.xml'

oscar_api = 'https://oscardepl.wmo.int/surface/rest/api/wmd/upload'


with open(file) as r:
    xml = r.read()
    res = requests.post(url=oscar_api, data=xml, headers={'X-WMO-WMDR-Token': token})
    print(res)

<Response [200]>


In [3]:
from pyoscar import OSCARClient
token = 'bea1a424-eee9-4edc-8467-c5b9497d2a6c'
file = '/home/sdanioth/Documents/git/oscar-catalogue/oscar-facility-catalogue/test_survey_2_KEN_234.xml'


client = OSCARClient(api_token=token)
print(client)

with open(file) as fh:
    data = fh.read()

response = client.upload(data)
print(response)

{'id': 691498, 'xmlStatus': 'VALID_XML_WITH_ERRORS_OR_WARNINGS', 'logs': 'The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).\n# Facility with identifier "0-404-300-TestSurvey2"\nREF_6: The selected identification process is using the ID attribute. No value found for the ID attribute of the element "deployment/Deployment" with beginPosition = "undefined". The "deployment" is discarded.\n'}


In [4]:
class FacilityCatalogue:
    """
    Read flat file, convert to WMDR XML file, upload to OSCAR/Surface.

    Read flat file containing facility catalogue entries, 
    convert to WMDR XML file, upload to OSCAR/Surface.
    """

    @classmethod
    def __init__(self, config):
        """
        Constructor

        Parameters
        ----------
        config : str
            Full path to configuration file.

        Returns
        -------
        None.

        """
        try:                        
            with open(os.path.abspath(config), "r") as f:
                config = yaml.safe_load(f)
                f.close()

            # setup logging
            for h in logging.root.handlers[:]:
                logging.root.removeHandler(h)
            logdir = os.path.expanduser(config['logging'])
            os.makedirs(logdir, exist_ok=True)
            logfile = '%s.log' % time.strftime('%Y%m%d')
            self.logfile = os.path.join(logdir, logfile)
            self.logger = logging.getLogger(__name__)
            logging.basicConfig(level=logging.DEBUG,
                                format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                                datefmt='%y-%m-%d %H:%M:%S',
                                filename=str(self.logfile),
                                filemode='a')
            
            self.source = os.path.expanduser(config['source'])
            self.target = os.path.expanduser(config['target'])
            self.template = config['template'] 
            self.header = { 'dtm': time.strftime("%Y-%m-%dT%H:%M:%S"), 
                    'individualName': config['individualName'],
                            'organisationName': config['organisationName'],
                            'electronicMailAddress': config['organisationName'] }

            self.upload = config['upload']
            self.proxies = config['proxies']
            self.token = config['token']
            
        except Exception as err:
            self.logger.error('Error setting up FacilityCatalogue', err)



    @classmethod
    def csv2wmdr(self, verbose=True):
        """
        Read a flat CSV file, cast to WMDR XML file, save as target

        Parameters
        ----------
        verbose : bln, optional
            Should log file be written? The default is True.

        Returns
        -------
        filename with full path.

        """
        try:
            sanitize = lambda x: html.escape(x) if len(x) > 0 else 'unknown'

            # read jinja template
            templateLoader = jinja2.FileSystemLoader(searchpath="./")
            templateEnv = jinja2.Environment(loader=templateLoader)
            template = templateEnv.get_template(self.template)

            # read source file and convert to list of dictionaries, one for
            # each combination of manufacturer, model, variable
            with open(self.source) as f:
                reader = csv.DictReader( f )
                files = []                
                for row in reader:
                    var = 'ObservedVariableAtmosphere'
                    d1 = { key: sanitize(row[key]) for key in row.keys() if key != var }
                    for ele in row[var].split(','):
                        facility = {var: ele.strip(), 'uuid': uuid.uuid4()} 
                        facility.update(d1)
                                   
                        # generate XML file
                        xml = template.render(header=self.header, facility=facility)
                        file = os.path.join(self.target, "%s %s %s.xml" % (facility['Name'], facility['Territory'], facility['ObservedVariableAtmosphere']))
                        file = file.replace(" ", "_")
                        files.append(file)                    
                        with open(file, 'w') as f:
                            f.write(xml)
                            f.close()
                        if verbose:
                            self.logger.info("XML file saved to " + file)
                
            return(files)

        except Exception as err:
            self.logger.error(err)
            return(err)
        
        

if __name__ == '__main__':
    config = os.path.join(os.getcwd(), 'config.yaml')
    facility_catalogue = FacilityCatalogue(config)
    facility_catalogue.csv2wmdr()
    

In [15]:
config = os.path.join(os.getcwd(), 'config.yaml')

with open(os.path.abspath(config), "r") as f:
    config = yaml.safe_load(f)
    f.close()

source = os.path.expanduser(config['source'])
target = os.path.expanduser(config['target'])
template = config['template'] 
header = { 'dtm': time.strftime("%Y-%m-%dT%H:%M:%S"),'individualName': config['individualName'],'organisationName': config['organisationName'],'electronicMailAddress': config['organisationName'] }

upload = config['upload']
proxies = config['proxies']
token = config['token']
